# Convert YOLO .pt model to metal core

In [3]:
CHECKPOINT = "full_trained.pt"

## Old

Run `yolo export model=model_checkpoints/yolo_classifier1.pt format=onnx opset=12` in Terminal

In [4]:
import coremltools as ct
import onnx

# Load ONNX model
onnx_model = onnx.load(CHECKPOINT)

# Convert to Core ML
coreml_model = ct.converters.onnx.convert(
    onnx_model,
    minimum_ios_deployment_target="18",  # or "14" depending on your app
)

# Save Core ML model
coreml_model.save("YOLOv8.mlmodel")

AttributeError: module 'coremltools.converters' has no attribute 'onnx'

In [5]:
import onnx

# Load your ONNX model
model = onnx.load(CHECKPOINT)

# Get the input tensor info
input_all = [inp for inp in model.graph.input]

# Skip initializers (constants, weights)
initializer_names = {init.name for init in model.graph.initializer}
inputs = [i for i in input_all if i.name not in initializer_names]

# Print input shapes
for input_tensor in inputs:
    shape = []
    tensor_type = input_tensor.type.tensor_type
    for dim in tensor_type.shape.dim:
        if dim.dim_value > 0:
            shape.append(dim.dim_value)
        else:
            shape.append("?")  # dynamic dim
    print(f"Input name: {input_tensor.name}")
    print(f"Shape: {shape}")

Input name: images
Shape: [1, 3, 512, 512]


## New

Load YOLO model and extract torch model

In [5]:
from ultralytics import YOLO
import torch

model = YOLO(CHECKPOINT)
model.export(format="coreml", nms=True) #! Check for more options (Try optimize=True later)

Ultralytics 8.3.144 🚀 Python-3.10.12 torch-2.5.1+cu124 CPU (Intel Xeon E5-2623 v4 2.60GHz)


YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'full_trained.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) (1, 14, 5376) (5.2 MB)

CoreML: starting export with coremltools 8.3.0...


Tuple detected at graph output. This will be flattened in the converted model.
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 36.84 passes/s]


CoreML Pipeline: starting pipeline with coremltools 8.3.0...
CoreML Pipeline: pipeline success
CoreML: export success ✅ 7.8s, saved as 'full_trained.mlpackage' (5.1 MB)

Export complete (8.3s)
Results saved to /data22/stu235269/TinyML-MT/model_checkpoints
Predict:         yolo predict task=detect model=full_trained.mlpackage imgsz=512  
Validate:        yolo val task=detect model=full_trained.mlpackage imgsz=512 data=../../huggingface/ai_checkout/artificial_rotated_mult_back/artificial_rotated_mult_back.yaml  
Visualize:       https://netron.app


'full_trained.mlpackage'